In [158]:
import pandas as pd
import numpy as np

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer, PorterStemmer
import pandas as pd
import nltk

# Download VADER lexicon if not already downloaded
nltk.download('vader_lexicon')
# Download WordNet if not already downloaded
nltk.download('wordnet')
nltk.data.path.append("/path/to/nltk_data")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pande\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pande\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [159]:
data = pd.read_csv('Student Mental Wellness Survey (Responses) - Form responses 1.csv')

data

,Timestamp,Age,Which state are you from?,Gender,Grade Level/Year in School,What are you currently studying?,How often have you been feeling sad or hopeless in the past two weeks?,"How often have you been feeling nervous, anxious, or on edge in the past two weeks?",Have you experienced changes in your appetite or weight in the past two weeks?,"Have you had trouble sleeping (either difficulty falling asleep, staying asleep, or waking up too early) in the past two weeks?",Have you been feeling unusually fatigued or lacking in energy in the past two weeks?,Have you had trouble concentrating on things such as schoolwork or other activities in the past two weeks?,Have you had recurring thoughts of self harm in the past two weeks?,"Have you experienced any physical symptoms such as racing heart, trembling, sweating, or shortness of breath without any apparent cause in the past two weeks?",Have you noticed any changes in your academic performance or attendance due to emotional distress in the past two weeks?,Is there anything else you would like to share about your emotional well-being or any concerns you have?,Do you think you have depression or anxiety?,Have you ever been diagnosed with depression or anxiety by a healthcare professional?
0,05/03/2024 22:23:50,20,Odisha,Male,Bachelor's 3rd Year,Computer Science,Everyday,Often,Weight gain,Often,Everyday,Everyday,Everyday,Rarely,No change,Nothing,Yes,No
1,05/03/2024 22:31:08,69,Liquid,Male,Bachelor's 3rd Year,Computer Science,Everyday,Everyday,No change,Everyday,Everyday,Sometimes,Everyday,Everyday,No change,nope,Yes,Yes
2,13/03/2024 12:53:08,20,Haryana,Male,Bachelor's 3rd Year,BBA,Sometimes,Rarely,No change,No,No,No,No,No,No change,No,No,Yes
3,13/03/2024 12:54:03,21,UP,Male,Bachelor's 4th Year,CSE,No,No,No change,No,No,No,No,No,No change,NaN,Yes,Yes
4,13/03/2024 12:54:48,56,Delhi,Female,Bachelor's 2nd Year,BA,Rarely,Rarely,Increased appetite,Rarely,Rarely,Rarely,Rarely,Rarely,Slight decline,NaN,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,20/03/2024 13:40:32,22,Odisha,Female,Bachelor's 4th Year,Computer science engineering,Often,Often,Decreased appetite,Everyday,Often,Often,Sometimes,Rarely,Slight decline,I am very exhausted mentally and physically,Yes,No
170,20/03/2024 14:03:29,18,Bihar,Female,Bachelor's 1st Year,Mathematics and computing,Sometimes,Sometimes,Increased appetite,Rarely,No,Sometimes,No,No,Slight decline,NaN,No,No
171,20/03/2024 14:42:03,21,Odisha,Male,Bachelor's 4th Year,In job,Rarely,Sometimes,Decreased appetite,Sometimes,No,No,No,No,No change,NaN,No,No
172,20/03/2024 20:54:17,22,Odisha,Female,Bachelor's 4th Year,B.Tech CSE,Sometimes,Rarely,No change,No,No,Sometimes,No,No,Slight decline,.,No,No


In [160]:
data.columns = [f'X{i}' for i in range(1, len(data.columns) + 1)]
data = data.iloc[:, 1:]
data

,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
0,20,Odisha,Male,Bachelor's 3rd Year,Computer Science,Everyday,Often,Weight gain,Often,Everyday,Everyday,Everyday,Rarely,No change,Nothing,Yes,No
1,69,Liquid,Male,Bachelor's 3rd Year,Computer Science,Everyday,Everyday,No change,Everyday,Everyday,Sometimes,Everyday,Everyday,No change,nope,Yes,Yes
2,20,Haryana,Male,Bachelor's 3rd Year,BBA,Sometimes,Rarely,No change,No,No,No,No,No,No change,No,No,Yes
3,21,UP,Male,Bachelor's 4th Year,CSE,No,No,No change,No,No,No,No,No,No change,NaN,Yes,Yes
4,56,Delhi,Female,Bachelor's 2nd Year,BA,Rarely,Rarely,Increased appetite,Rarely,Rarely,Rarely,Rarely,Rarely,Slight decline,NaN,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,22,Odisha,Female,Bachelor's 4th Year,Computer science engineering,Often,Often,Decreased appetite,Everyday,Often,Often,Sometimes,Rarely,Slight decline,I am very exhausted mentally and physically,Yes,No
170,18,Bihar,Female,Bachelor's 1st Year,Mathematics and computing,Sometimes,Sometimes,Increased appetite,Rarely,No,Sometimes,No,No,Slight decline,NaN,No,No
171,21,Odisha,Male,Bachelor's 4th Year,In job,Rarely,Sometimes,Decreased appetite,Sometimes,No,No,No,No,No change,NaN,No,No
172,22,Odisha,Female,Bachelor's 4th Year,B.Tech CSE,Sometimes,Rarely,No change,No,No,Sometimes,No,No,Slight decline,.,No,No


In [161]:
print(data['X17'].value_counts())
data['X3'].unique()

X17
No                90
I am not sure.    47
Yes               37
Name: count, dtype: int64


array(['Odisha', 'Liquid', 'Haryana ', 'UP ', 'Delhi', 'Rajisthan',
       'haryana', 'Bihar', 'Up', 'Uttar pradesh ', 'Uttar Pradesh ',
       'Maharashtra ', 'Bihar ', 'Uttar pradesh', 'West Bengal ',
       'Maharastra ', '-', 'Madhya Pradesh ', 'Jharkhand ', 'Tamil Nadu ',
       'UP', 'Madhya pradesh ', 'Uttar Pradesh', 'Haryana', 'Odisha ',
       'Jharkhand', 'Rajasthan', 'Chhattisgarh', 'HARYANA ',
       'Chhattisgarh ', 'MP', 'Karnataka ', 'Maharashtra',
       'Not from India', 'West Bengal', 'Gujrat ', 'Delhi NCR',
       'Madhya Pradesh', 'UttarPradesh', 'uttar Pradesh', 'Assam',
       'maharashtra ', 'Karnataka', 'Goa', 'delhi ', 'mp', 'Mahrashtra'],
      dtype=object)

In [162]:
data.loc[:, 'X3'] = data['X3'].str.strip()
data['X3'].unique()

array(['Odisha', 'Liquid', 'Haryana', 'UP', 'Delhi', 'Rajisthan',
       'haryana', 'Bihar', 'Up', 'Uttar pradesh', 'Uttar Pradesh',
       'Maharashtra', 'West Bengal', 'Maharastra', '-', 'Madhya Pradesh',
       'Jharkhand', 'Tamil Nadu', 'Madhya pradesh', 'Rajasthan',
       'Chhattisgarh', 'HARYANA', 'MP', 'Karnataka', 'Not from India',
       'Gujrat', 'Delhi NCR', 'UttarPradesh', 'uttar Pradesh', 'Assam',
       'maharashtra', 'Goa', 'delhi', 'mp', 'Mahrashtra'], dtype=object)

In [163]:
data.loc[:, 'X3'] = data['X3'].str.strip().str.lower()
replacements = {'up': 'Uttar Pradesh', 'uttarpradesh': 'Uttar Pradesh', 'mp': 'Madhya Pradesh', 'delhi ncr': 'Delhi','mahrashtra':'Maharashtra','rajisthan' : 'Rajasthan'}
data.loc[:, 'X3'] = data['X3'].replace(replacements)

data.loc[:, 'X3'] = data['X3'].apply(lambda x: x.title())

data['X3'].value_counts()

X3
Odisha            34
Maharashtra       28
Madhya Pradesh    19
Bihar             18
Jharkhand         18
Uttar Pradesh     14
Haryana            7
Assam              7
Delhi              6
West Bengal        6
Chhattisgarh       4
Karnataka          3
Rajasthan          2
Maharastra         2
Not From India     1
Gujrat             1
-                  1
Tamil Nadu         1
Liquid             1
Goa                1
Name: count, dtype: int64

In [166]:
data['X16'] = data['X16'].astype(str)

sia = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


negative_responses = ['no', 'nope', 'nothing', 'na','nope.','nah','not at all','no.', ]


data['X16_sentiment'] = data['X16'].apply(lambda x: 0.0 if any(token in [stemmer.stem(lemmatizer.lemmatize(word)) for word in nltk.word_tokenize(x.lower())] for token in negative_responses) else sia.polarity_scores(x)['compound'] if isinstance(x, str) else 0.0)

columns_to_encode = [col for col in data.columns if col not in ['X2', 'X6', 'X17', 'X16', 'X16_sentiment']]
data_encoded = pd.get_dummies(data, columns=columns_to_encode, drop_first=False)

data = data_encoded.drop(columns=['X16','X6'])
data


,X2,X17,X16_sentiment,X3_-,X3_Assam,X3_Bihar,X3_Chhattisgarh,X3_Delhi,X3_Goa,X3_Gujrat,...,X14_Everyday,X14_No,X14_Often,X14_Rarely,X14_Sometimes,X15_No change,X15_Significant decline,X15_Slight decline,X18_No,X18_Yes
0,20,Yes,0.0000,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,True,False
1,69,Yes,0.0000,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,True
2,20,No,0.0000,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,True
3,21,Yes,0.0000,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,True
4,56,No,0.0000,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,22,Yes,-0.4201,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,True,False
170,18,No,0.0000,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,True,True,False
171,21,No,0.0000,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,True,False
172,22,No,0.0000,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,True,True,False


In [167]:
data['X16_sentiment'].value_counts()

X16_sentiment
 0.0000    154
-0.4215      3
-0.7430      2
-0.2732      1
-0.1027      1
-0.2500      1
 0.3818      1
-0.1832      1
 0.3164      1
 0.3400      1
 0.8715      1
 0.4310      1
 0.3382      1
 0.5837      1
-0.8889      1
 0.6249      1
-0.5106      1
-0.4201      1
Name: count, dtype: int64

In [168]:
# Save DataFrame to CSV file
data.to_csv('data_with_sentiment.csv', index=False)


In [169]:
X = data.drop(columns=['X17'])  
y = data['X17']  

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X, y)

with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(rf_classifier, file)

In [170]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Encode string labels to integer values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define your neural network architecture
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # Output layer with softmax activation for multiclass classification
])

# Compile the model
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the neural network
nn_model.fit(X, y_encoded, epochs=300, batch_size=50, validation_split=0.2)  # Adjust epochs, batch_size, and validation_split as needed

#print accuracy
accuracy = nn_model.evaluate(X, y_encoded)[1]
print("Neural Network Accuracy:", accuracy)





Epoch 1/300


c:\Ananconda\envs\SELF\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.2753 - loss: 2.0214 - val_accuracy: 0.2286 - val_loss: 1.6347
Epoch 2/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2728 - loss: 1.4898 - val_accuracy: 0.2286 - val_loss: 1.3138
Epoch 3/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2714 - loss: 1.1992 - val_accuracy: 0.1429 - val_loss: 1.1675
Epoch 4/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4485 - loss: 1.0719 - val_accuracy: 0.4286 - val_loss: 1.1164
Epoch 5/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5548 - loss: 1.0407 - val_accuracy: 0.4571 - val_loss: 1.1085
Epoch 6/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5237 - loss: 1.0481 - val_accuracy: 0.4571 - val_loss: 1.1198
Epoch 7/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5562 - loss: 1.0142 - val_accuracy: 0.4571 - val_loss: 1.1372
Epoch 8/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5512 - loss: 0.9989 - val_accuracy: 0.4571 - val_loss: 1.1460
Epoc